# Natural Language Processing on Amazon Fine Food Reviews
## Assignment 2

**Aadith Sukumar**<br>
**AIML A1**<br>
**21070126003**<br>

Data Source: [Kaggle](https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews?resource=download)

Questions:
1. Use the same dataset from Assignment 1.

2. DO FOLLOWING Preprocessing STEPS using NLTK or SpaCy:
    1. Remove empty rows. Remove duplicates.
    2. Tokenization, Lemmatization
    3. Data Cleansing: Remove stopwords, remove symbols,remove emojis,     remove URLS, remove http tags, remove excess whitespaces.
    4. Lower the strings, replace abbreviations, fix contractions.

3. Use following as range of values for input variables:
    1. For 1st set of results use:
      1. batch_size = 4
      2. max_sequence_length = 50
      2. embedding_dim = 50
      2. max_words = 10000
      2. lstm_units = 32
     
    2. For 2st set of results use:
      1. batch_size = 8
      1. max_sequence_length = 30
      1. embedding_dim = 30
      1. max_words = 25000
      1. lstm_units = 32

4. Use following LSTM:
    1. Single Layer of LSTM for 1st set of results
    2. Two Layers of LSTM for 2nd set of results

5. Compare the results using Classification Report.

In [2]:
!nvidia-smi

Sat Sep  2 11:04:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.41       Driver Version: 527.41       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000   WDDM  | 00000000:2D:00.0  On |                  Off |
| 41%   38C    P8    10W / 140W |    296MiB / 16376MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


## Importing Libraries

In [4]:
import pandas as pd
import nltk
# nltk.download("all")
import spacy

from spacy.lang.en.stop_words import STOP_WORDS
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')

## 1. Loading The Dataset 

In [26]:
data = pd.read_csv("Reviews.csv")
data.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [27]:
print("Shape of data: ",data.shape)
data.info()

Shape of data:  (568454, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


## Sampling Data for Faster Processing

In [32]:
# sampling data to get 10000 rows
data = data.sample(n=10000)
data.head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
381574,381575,B0035R36QU,A187VKK7VWKQ99,Food Snob,2,9,1,1317945600,Smells and Taste like Fish,The picture is NOT the color of the salt. It s...
60005,60006,B000E48IPG,A15V9LJDW699AQ,Justanotherguy,0,0,5,1283817600,Part of a Healthy Breakfast,Some students of nutrition believe that it is ...
365964,365965,B008IY5EGU,AMLU3405N8CUI,Gretchen F.,14,16,1,1344902400,Sticking with original Pop Chips,I am a HUGE fan of Pop Chips and buy them regu...
456899,456900,B001EO5S2Q,A2E3WMF9RWW2X2,K. Duvall,0,0,5,1203638400,Great crunch and tasty.,These were very good. Good crunch and low sug...
412626,412627,B004725ZX4,A3EA67H2C3R6VF,"Java ""Mom of one""",0,0,5,1298937600,Awesome,Everyone who comes to my house LOVES these cra...
277462,277463,B000VK8AVK,A2UEO5XR3598GI,Rich K,2,2,5,1296691200,"YUMMY, YUMMY a little bit of love for your Tummy",Hey!! these things taste good. At first I tho...
391411,391412,B000F9XB7K,AD6XWLNH2X8S7,WILLOBIE,6,6,5,1283990400,Still better than the imitators.,My local stores have not been stocking Red Ova...
341933,341934,B000ESJR20,A316RLHHO9Y24T,myoho guy,52,53,5,1219363200,Mount Hagen vs Taster's Choice,"I like freshly-ground, brewed coffee, but some..."
319899,319900,B0018QIPS4,A234HXDATOAYEY,"Karen Baker ""city mama""",0,0,5,1280793600,Addictive!,I started buying these as a non-messy way for ...
364465,364466,B000PDY3HI,A1VYU7DCJ3GA9E,Katy,0,0,5,1338249600,LOVE this popcorn!!!,This is a GREAT item -- very easy to use and y...


In [33]:
print("Shape of data: ",data.shape)
data.info()

Shape of data:  (10000, 10)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 381574 to 32509
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Id                      10000 non-null  int64 
 1   ProductId               10000 non-null  object
 2   UserId                  10000 non-null  object
 3   ProfileName             10000 non-null  object
 4   HelpfulnessNumerator    10000 non-null  int64 
 5   HelpfulnessDenominator  10000 non-null  int64 
 6   Score                   10000 non-null  int64 
 7   Time                    10000 non-null  int64 
 8   Summary                 9999 non-null   object
 9   Text                    10000 non-null  object
dtypes: int64(5), object(5)
memory usage: 859.4+ KB


## 2. Preprocessing Data

### a. Lemmatization with Spacy & Tokenization

In [9]:
!spacy download en_core_web_sm

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 386, in _make_request
    self._validate_conn(conn)
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1042, in _validate_conn
    conn.connect()
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connection.py", line 419, in connect
    self.sock = ssl_wrap_socket(
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\ProgramData\Anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\ProgramData\Anaconda3\lib\ssl.py", line 501, in wrap_socket
    return self.sslsocket_class._cr

In [11]:
nlp = spacy.load('en_core_web_sm')
data['Text'] = data['Text'].apply(lambda x: nlp(x))

In [12]:
# Tokenization
data['Text'] = data['Text'].apply(lambda x: [item.text for item in x])
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
406847,406848,B004H4QZDQ,A2M5VD2YDK2LQL,Robert L. Spencer,0,0,4,1319155200,Great soup but nothing like the picture,"[I, really, likd, the, soup, ,, but, it, was, ..."
166456,166457,B000FA39GW,A13NIU1KP9TI5P,"Ali ""MI Matriarch""",0,0,5,1269388800,Childhood Favorite,"[These, cookies, are, one, of, those, childhoo..."


### b. Stop Words, Symbols, URLs and Special Characters Removal

In [13]:
# removing stop words
data['Text'] = data['Text'].apply(lambda x: [item for item in x if item not in STOP_WORDS])
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
406847,406848,B004H4QZDQ,A2M5VD2YDK2LQL,Robert L. Spencer,0,0,4,1319155200,Great soup but nothing like the picture,"[I, likd, soup, ,, liquid, vegetables, ,, They..."
166456,166457,B000FA39GW,A13NIU1KP9TI5P,"Ali ""MI Matriarch""",0,0,5,1269388800,Childhood Favorite,"[These, cookies, childhood, favorites, help, p..."


In [14]:
# remove symbols and special characters
data['Text'] = data['Text'].apply(lambda x: [item for item in x if item.isalpha()])
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
406847,406848,B004H4QZDQ,A2M5VD2YDK2LQL,Robert L. Spencer,0,0,4,1319155200,Great soup but nothing like the picture,"[I, likd, soup, liquid, vegetables, They, look..."
166456,166457,B000FA39GW,A13NIU1KP9TI5P,"Ali ""MI Matriarch""",0,0,5,1269388800,Childhood Favorite,"[These, cookies, childhood, favorites, help, a..."


In [15]:
# urls
data['Text'] = data['Text'].apply(lambda x: [item for item in x if "http" not in item])
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
406847,406848,B004H4QZDQ,A2M5VD2YDK2LQL,Robert L. Spencer,0,0,4,1319155200,Great soup but nothing like the picture,"[I, likd, soup, liquid, vegetables, They, look..."
166456,166457,B000FA39GW,A13NIU1KP9TI5P,"Ali ""MI Matriarch""",0,0,5,1269388800,Childhood Favorite,"[These, cookies, childhood, favorites, help, a..."


### Lowering the string, fix contractions

In [16]:
contraction_mapping = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "I'd": "I would",
    "I'll": "I will",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it's": "it is",
    "let's": "let us",
    "mustn't": "must not",
    "shan't": "shall not",
    "she'd": "she would",
    "she'll": "she will",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they would",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "we'd": "we would",
    "we'll": "we will",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "where's": "where is",
    "who'd": "who would",
    "who'll": "who will",
    "who're": "who are",
    "who's": "who is",
    "who've": "who have",
    "won't": "will not",
    "wouldn't": "would not",
    "you'd": "you would",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have",
    "'s": " is",
    "'ll": " will",
    "'d": " would",
    "'ve": " have",
    "'re": " are",
    "n't": " not",
}

data['Text'] = data['Text'].apply(lambda x: [contraction_mapping.get(item, item) for item in x])
data['Text'] = data['Text'].apply(lambda x: [item.lower() for item in x])
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
406847,406848,B004H4QZDQ,A2M5VD2YDK2LQL,Robert L. Spencer,0,0,4,1319155200,Great soup but nothing like the picture,"[i, likd, soup, liquid, vegetables, they, look..."
166456,166457,B000FA39GW,A13NIU1KP9TI5P,"Ali ""MI Matriarch""",0,0,5,1269388800,Childhood Favorite,"[these, cookies, childhood, favorites, help, a..."


## 3. Preparing LSTM Models

### For 1st set of results use:
    1. batch_size = 4
    2. max_sequence_length = 50
    3. embedding_dim = 50
    4. max_words = 10000
    5. lstm_units = 32
    6. Single Layer


In [16]:
!pip install keras.preprocessing

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/42.6 kB ? eta -:--:--
     ---------------------------------------- 42.6/42.6 kB 2.0 MB/s eta 0:00:00


In [34]:
# LSTM model with batch size 4, max sequence length 50, embedding dim 50, max words 10000, lstm units 32

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [35]:
max_fatures = 10000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Text'].values)
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X)
y = pd.get_dummies(data['Score']).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [36]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(6700, 1234) (3300, 1234)
(6700, 5) (3300, 5)


In [37]:
model_one = Sequential()
model_one.add(Embedding(max_fatures, 50, input_length=X.shape[1]))
model_one.add(SpatialDropout1D(0.4))
model_one.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model_one.add(Dense(5, activation='softmax'))
model_one.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_one.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1234, 50)          500000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 1234, 50)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 32)                10624     
                                                                 
 dense (Dense)               (None, 5)                 165       
                                                                 
Total params: 510,789
Trainable params: 510,789
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
model_one.fit(X_train, y_train, epochs=20, batch_size=4, callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.01, patience=7, restore_best_weights=True)])

Epoch 1/20
1675/1675 [==============================] - 460s 275ms/step - loss: 0.9347 - accuracy: 0.6697
Epoch 2/20
1675/1675 [==============================] - 463s 276ms/step - loss: 0.7574 - accuracy: 0.7184
Epoch 3/20
1675/1675 [==============================] - 464s 277ms/step - loss: 0.6211 - accuracy: 0.7685
Epoch 4/20
1675/1675 [==============================] - 462s 276ms/step - loss: 0.5104 - accuracy: 0.8082
Epoch 5/20
1675/1675 [==============================] - 464s 277ms/step - loss: 0.4309 - accuracy: 0.8446
Epoch 6/20
1675/1675 [==============================] - 465s 278ms/step - loss: 0.3586 - accuracy: 0.8687
Epoch 7/20
1675/1675 [==============================] - 474s 283ms/step - loss: 0.3084 - accuracy: 0.8890
Epoch 8/20
1675/1675 [==============================] - 469s 280ms/step - loss: 0.2521 - accuracy: 0.9112
Epoch 9/20
1675/1675 [==============================] - 470s 281ms/step - loss: 0.2090 - accuracy: 0.9304
Epoch 10/20
1675/1675 [=======================

In [23]:
# save model
model_one.save('model_one.h5')

### For 2nd set of results use:
    1. batch_size = 8
    2. max_sequence_length = 30
    3. embedding_dim = 30
    4. max_words = 25000
    5. lstm_units = 32
    6. Two Layers

In [38]:
# LSTM model with batch size 8, max sequence length 30, embedding dim 30, max words 25000, lstm units 32

max_features = 25000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['Text'].values)
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X)
y = pd.get_dummies(data['Score']).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [40]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(6700, 1279) (3300, 1279)
(6700, 5) (3300, 5)


In [41]:
model_two = Sequential()
model_two.add(Embedding(max_features, 30, input_length=X.shape[1]))
model_two.add(SpatialDropout1D(0.4))
model_two.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model_two.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model_two.add(Dense(5, activation='softmax'))
model_two.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_two.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1279, 30)          750000    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 1279, 30)         0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 1279, 32)          8064      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 766,549
Trainable params: 766,549
Non-trainable params: 0
________________________________________________

In [ ]:
model_two.fit(X_train, y_train, epochs=20, batch_size=8, callbacks=[EarlyStopping(monitor='loss', min_delta=0.01, patience=7, restore_best_weights=True)])

Epoch 1/20
838/838 [==============================] - 614s 729ms/step - loss: 0.3323 - accuracy: 0.8721
Epoch 2/20
838/838 [==============================] - 608s 726ms/step - loss: 0.2732 - accuracy: 0.8996
Epoch 3/20
407/838 [=============>................] - ETA: 5:10 - loss: 0.2557 - accuracy: 0.9057

In [28]:
model_two.save('model_two.h5')

---

## Evaluation and Comparison

In [140]:
# load the model
from keras.models import load_model
from sklearn.model_selection import train_test_split

In [141]:
data = pd.read_csv("Reviews.csv")
data = data.sample(n=10000)

In [142]:
max_fatures = 10000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Text'].values)
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X, maxlen=433)
y = pd.get_dummies(data['Score']).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [143]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(6700, 433) (3300, 433)
(6700, 5) (3300, 5)


In [144]:
model_one = load_model('model_one.h5', compile=False)
model_one.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [145]:
loss_m1, accuracy_m1 = model_one.evaluate(X_test, y_test)
print("loss of model 1: ", loss_m1, "accuracy of model 1: ", accuracy_m1)

104/104 [==============================] - 4s 29ms/step - loss: 3.6179 - accuracy: 0.5642
loss of model 1:  3.6178691387176514 accuracy of model 1:  0.564242422580719


In [146]:
predictions_m1 = model_one.predict(X_test)
print(predictions_m1)

104/104 [==============================] - 3s 28ms/step
[[1.0638024e-06 3.2071828e-08 8.2952107e-07 7.3840516e-07 9.9999738e-01]
 [2.0340972e-06 4.9299075e-08 6.3931896e-07 2.2569320e-06 9.9999499e-01]
 [1.6369984e-02 4.5352869e-02 6.3043994e-01 1.7141470e-01 1.3642256e-01]
 ...
 [4.4416352e-05 1.3472097e-06 1.5721776e-05 1.7671134e-04 9.9976176e-01]
 [1.3842887e-06 2.5430678e-08 1.1531790e-06 2.3747602e-06 9.9999511e-01]
 [8.9920559e-06 4.2243559e-07 2.3523769e-05 4.5122990e-05 9.9992192e-01]]


In [147]:
max_features = 25000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['Text'].values)
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X, maxlen=446)
y = pd.get_dummies(data['Score']).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [148]:
model_two = load_model('model_two.h5', compile=False)
model_two.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [149]:
loss_m2, accuracy_m2 = model_two.evaluate(X_test, y_test)
print("loss of model 2: ", loss_m2, "accuracy of model 2: ", accuracy_m2)

104/104 [==============================] - 6s 54ms/step - loss: 2.8901 - accuracy: 0.4782
loss of model 2:  2.8901052474975586 accuracy of model 2:  0.4781818091869354


In [150]:
predictions_m2 = model_two.predict(X_test)
print(predictions_m2)

104/104 [==============================] - 6s 54ms/step
[[1.4336625e-05 1.4083895e-06 8.1047829e-06 7.2286732e-04 9.9925321e-01]
 [2.2305366e-04 1.2349457e-04 1.5455611e-03 7.5510740e-02 9.2259717e-01]
 [9.0441294e-03 3.8708073e-01 5.8392346e-01 1.8992374e-02 9.5926132e-04]
 ...
 [2.9513609e-05 4.5135880e-06 3.2139142e-05 2.4919466e-03 9.9744189e-01]
 [1.4858582e-05 1.9242018e-06 1.9953321e-05 1.4632988e-03 9.9849999e-01]
 [1.7404795e-05 2.1904339e-06 1.5458783e-05 1.3657188e-03 9.9859923e-01]]


In [151]:
# compare results using Classification Report
from sklearn.metrics import classification_report

In [152]:
print(classification_report(y_test.argmax(axis=1), predictions_m1.argmax(axis=1)))
print(classification_report(y_test.argmax(axis=1), predictions_m2.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.07      0.02      0.04       295
           1       0.15      0.02      0.03       175
           2       0.06      0.02      0.03       244
           3       0.16      0.09      0.11       458
           4       0.64      0.85      0.73      2128

    accuracy                           0.56      3300
   macro avg       0.22      0.20      0.19      3300
weighted avg       0.45      0.56      0.49      3300

              precision    recall  f1-score   support

           0       0.05      0.01      0.01       295
           1       0.06      0.02      0.03       175
           2       0.02      0.01      0.02       244
           3       0.13      0.22      0.17       458
           4       0.64      0.69      0.66      2128

    accuracy                           0.48      3300
   macro avg       0.18      0.19      0.18      3300
weighted avg       0.44      0.48      0.45      3300



---
END OF FILE
---